# Notebook 2

[Download the notebook](notebook2.zip)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/gjouvet/glacier-tutorials/blob/main/notebooks/demo.ipynb)

## Goals of this notebook

   - Get familiar with model IGM
   - Understand the parameters, set-up your model
   - Run IGM examples
   - Custumization, run IGM for your own glacier

## What is IGM?

Here we will use the glacier evolution model developed at UNIL -- the Instructed Glacier Model.
 
The IGM is an open-source Python package, which permits to simulate 3D glacier evolution accounting for the coupling between ice thermo-dynamics, surface mass balance, and mass conservation. IGM is especially computationally efficient thanks to the TensorFlow library, mathematical operations are fully-vectorized. This permits tremendous speed-ups on [GPU](https://en.wikipedia.org/wiki/Graphics_processing_unit) (Graphics Processing Units), which are devices made of many processors, and can therefore perform highly-parallelized computations.

All the documentation can be found at [https://igm-model.org/](https://igm-model.org/).

## How to install IGM?

Running IGM consists of running a python script `igm_run`, which is made of functions of the `igm` python package.  First of all, one needs to install igm as follows:

In [ ]:
# Installation of the igm package directly from the github repository, this may take a while
!pip install git+https://github.com/instructed-glacier-model/igm.git@main

## First run with IGM

### Folder exploration

Let us explore the directory with the following comand:

In [ ]:
!tree

You will see the following files :

```bash
.
├── data 
├── experiment
│   ├── params_0.yaml
│   ├── params_1.yaml
│   └── params_2.yaml  
└── user
    ├── code 
    └── conf 
```

The important here are the folder `data`, which will contain all the **input** data to run the model, the parameters file in folder **experiment**. The folder **user** can serve to custumize some function (such as SMB function).

### Parameter file exploration

If you look at the parameter file, you will see the following files, whcih shows all the input, process, and output modules along with all their parameters.

Below, you may notice that IGM :
   - first loads a certain number of inputs, processes, and output modules.
   - Second define parameters associated with each module that deviates from default ones.

IGM is organized module-wise : as detailled here : https://igm-model.org/latest/modules/introduction/

In the example below (taken from IGM's exampled `aletsch-basic`), IGM loads data with the `local` module, proceed to a simulation from 1900 to 2000 updating surface mass balance (`smb`), iceflow, time, and ice thickness. Then, it writes the results in both a local file (netcdf file), and in form of plots.


```yaml title="params.yaml"
# @package _global_

defaults:
  - override /inputs: 
    - local
  - override /processes: 
    - smb_simple
    - iceflow
    - time
    - thk
  - override /outputs: 
    - local
    - plot2d

inputs:
  local:
     filename: input.nc

processes:
  smb_simple:
    array:
      - ["time", "gradabl", "gradacc", "ela", "accmax"]
      - [1900, 0.009, 0.005, 2800, 2.0]
      - [2000, 0.009, 0.005, 2900, 2.0]
      - [2100, 0.009, 0.005, 3300, 2.0]
  time:
    start: 1900.0
    end: 2000.0
    save: 10.0
    
outputs:
  plot2d:
    live: False
```

### Runing IGM (`params_0`)

Now, we can run IGM by simply run the command `igm_run +experiment=params` (or replace `params` with the actual name of your parameter file) in the corresponding folder to execute the example.

In [ ]:
!igm_run +experiment=params_0

### Explore the output

Each IGM run will then generate a folder with the day as the parent folder and the time as a subfolder. For instance, after a few runs we could end up with the following structure

```bash
outputs
  └───2025-04-07
      ├─── 12-06-29 
      └─── 12-07-38 
``` 


### Runing IGM (`params_1`)

The above example is somewhat limited

Copy and paste the following YAML parameter file (name it `params.yaml`), place it in a folder named `experiment`, and then run the command `igm_run +experiment=params`. This will model the Glacier with RGI ID `RGI60-11.01450` (the Great Aletsch Glacier, Switzerland) from 2020 to 2100, assuming a temperature increase of 4 degrees by 2100 relative to 1960–1990. 

The run will:
1. Use the `oggm_shop` module to download all necessary data via OGGM.
2. Execute a forward model combining the OGGM-based SMB model and the IGM-based ice flow model.
3. Write and plot the results in real time.

!!! warning

    This setup is provided as an example and has not been validated against historical data. It should not be interpreted as a scientifically accurate simulation. After running this example, you can explore different glaciers by selecting a different ID (refer to the [GLIMS Viewer](https://www.glims.org/maps/glims)), modify parameters, or experiment with additional modules.
